# Encontro 13: Medidas de Centralidade

Importando a biblioteca:

In [1]:
import sys
sys.path.append('..')

from random import choice
from itertools import permutations

import pandas as pd
import networkx as nx

import socnet as sn

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
sn.node_size = 10
sn.node_color = (255, 255, 255)

sn.edge_width = 1
sn.edge_color = (192, 192, 192)

sn.node_label_position = 'top center'

In [3]:
g = sn.load_graph('Renaissance.gml', has_pos=True)

sn.show_graph(g, nlab=True)

In [4]:
def set_geodesic_successors(g, s, t):
    for n in g.nodes:
        g.nodes[n]['geodesic_successors'] = set()

    for p in nx.all_shortest_paths(g, s, t):
        for i in range(len(p) - 1):
            g.nodes[p[i]]['geodesic_successors'].add(p[i + 1])

In [5]:
# Pense que o atributo 'passages' abaixo indica quantas
# vezes um fluxo já passou por um nó ou por uma aresta.

def random_geodesic_successor(g, n):
    return choice([m for m in g.nodes[n]['geodesic_successors']])

def random_path_successor(g, n):
    return choice([m for m in g.neighbors(n) if g.nodes[m]['passages'] == 0])

def random_trail_successor(g, n):
    return choice([m for m in g.neighbors(n) if g.edges[n, m]['passages'] == 0])

def random_walk_successor(g, n):
    return choice([m for m in g.neighbors(n)])

In [6]:
option = "geodesic"
isTransf = True

In [7]:
def simulate_single_flow(g, s, t, option, isTransf):
    # Inicializa o atributo 'passages' de cada nó.
    for n in g.nodes:
        g.nodes[n]['passages'] = 0
    g.nodes[s]['passages'] = 1

    # Inicializa o atributo 'passages' de cada aresta.
    for n, m in g.edges:
        g.edges[n, m]['passages'] = 0

    # Inicializa s como o único dono do insumo.
    for n in g.nodes:
        g.nodes[n]['owner'] = False
    g.nodes[s]['owner'] = True

    # Simula o fluxo, contando o número total de passos.

    steps = 0

    while True:
        # O conjunto reached representa todos os nós
        # que o fluxo consegue alcançar no passo atual.
        reached = set()

        # Verifica cada um dos donos atuais do insumo.

        owners = [n for n in g.nodes if g.nodes[n]['owner']]

        for n in owners:

            try:
                # Escolhe aleatoriamente um dos sucessores, escolhendo tipo = geodesic, path, trail or walk
                if option == "path":
                    m = random_path_successor(g, n)
                elif option == "trail":
                    m = random_trail_successor(g, n)
                elif option == "walk":
                    m = random_walk_successor(g, n)
                else:
                    m = random_geodesic_successor(g, n)
                    
            except IndexError:
                continue

            if isTransf == True:
                # Deixa de ser dono do insumo.
                g.nodes[n]['owner'] = False

            # Incrementa o atributo 'passages' do nó.
            g.nodes[m]['passages'] += 1

            # Incrementa o atributo 'passages' da aresta.
            g.edges[n, m]['passages'] += 1

            # Registra que consegue alcançar esse nó.
            reached.add(m)

        # Todo nó alcançado passa a ser dono do insumo.

        for n in reached:
            g.nodes[n]['owner'] = True

        # Isso conclui o passo atual da simulação.
        steps += 1

        # Se o passo alcançou t, chegamos ao fim da simulação.
        # Ela foi bem-sucedida: devolvemos o número de passos.
        if t in reached:
            return steps

        # Se o passo não alcançou ninguém, chegamos ao fim da
        # simulação. Ela não foi bem-sucedida: devolvemos -1.
        if not reached:
            return -1

In [8]:
def simulate_successful_flow(g, s, t, option, isTransf):
    set_geodesic_successors(g, s, t)

    while True:
        steps = simulate_single_flow(g, s, t, option, isTransf)

        if steps != -1:
            return steps

In [9]:
def simulate_all_flows(g, option, isTransf):
    for n in g.nodes:
        g.nodes[n]['closeness'] = 0
        g.nodes[n]['betweenness'] = 0

    for s, t in permutations(g.nodes, 2):
        steps = simulate_successful_flow(g, s, t, option, isTransf)

        g.nodes[s]['closeness'] += steps
        for n in g.nodes:
            if n != s and n != t:
                g.nodes[n]['betweenness'] += g.nodes[n]['passages']

    # Normalizações necessárias para comparar com os
    # resultados analíticos. Não é preciso entender.
    for n in g.nodes:
        g.nodes[n]['closeness'] = (g.number_of_nodes() - 1) / g.nodes[n]['closeness']
        g.nodes[n]['betweenness'] /= (g.number_of_nodes() - 1) * (g.number_of_nodes() - 2)

In [10]:
def simulacoes(g, option, isTransf):
    TIMES = 100

    for n in g.nodes:
        
        g.nodes[n]['lista_closeness'] = []
        g.nodes[n]['lista_betweenness'] = []
        g.nodes[n]['mean_closeness'] = 0
        g.nodes[n]['mean_betweenness'] = 0

    for _ in range(TIMES):
        simulate_all_flows(g, option, isTransf)

        for n in g.nodes:
            g.nodes[n]['lista_closeness'].append(g.nodes[n]['closeness'])
            g.nodes[n]['lista_betweenness'].append(g.nodes[n]['betweenness'])
            g.nodes[n]['mean_closeness'] += g.nodes[n]['closeness']
            g.nodes[n]['mean_betweenness'] += g.nodes[n]['betweenness']
            
    for n in g.nodes:
        g.nodes[n]['mean_closeness'] /= TIMES
        g.nodes[n]['mean_betweenness'] /= TIMES
        
    return {n: (g.nodes[n]['lista_closeness'], g.nodes[n]['lista_betweenness'], g.nodes[n]['mean_closeness'], g.nodes[n]['mean_betweenness']) for n in g.nodes}

In [11]:
geoTrue = simulacoes(g, "geodesic", True)
geoFalse = simulacoes(g, "geodesic", False)
pathTrue = simulacoes(g, "path", True)
pathFalse = simulacoes(g, "path", False)
trailTrue = simulacoes(g, "trail", True)
trailFalse = simulacoes(g, "trail", False)
walkTrue = simulacoes(g, "walk", True)
walkFalse = simulacoes(g, "walk", False)

#### Hipótese alternativa
Quando consideramos outros tipos de trajetória e outros tipos de difusão, os nós com maior closeness simulado e betweenness simulado não são necessariamente os nós com maior closeness e betweenness segundo as fórmulas clássicas. (que correspondem ao uso de geodésica e transferência na simulação)

In [12]:
from scipy import stats
a = 0
for n in g.nodes:
    g.nodes[n]['pvalor_closeness'] = []
    g.nodes[n]['pvalor_betweenness'] = []
    #Comparando default (geodésica transferência) com geodésica duplicação
    g.nodes[n]['pvalor_closeness'].append(stats.ttest_ind(geoTrue[n][0],geoFalse[n][0])[1])
    g.nodes[n]['pvalor_betweenness'].append(stats.ttest_ind(geoTrue[n][1],geoFalse[n][1])[1])
    #Comparando default (geodésica transferência) com path transferência
    g.nodes[n]['pvalor_closeness'].append(stats.ttest_ind(geoTrue[n][0],pathTrue[n][0])[1])
    g.nodes[n]['pvalor_betweenness'].append(stats.ttest_ind(geoTrue[n][1],pathTrue[n][1])[1])
    #Comparando default (geodésica transferência) com path duplicação
    g.nodes[n]['pvalor_closeness'].append(stats.ttest_ind(geoTrue[n][0],pathFalse[n][0])[1])
    g.nodes[n]['pvalor_betweenness'].append(stats.ttest_ind(geoTrue[n][1],pathFalse[n][1])[1])
    #Comparando default (geodésica transferência) com trail transferência
    g.nodes[n]['pvalor_closeness'].append(stats.ttest_ind(geoTrue[n][0],trailTrue[n][0])[1])
    g.nodes[n]['pvalor_betweenness'].append(stats.ttest_ind(geoTrue[n][1],trailTrue[n][1])[1])
    #Comparando default (geodésica transferência) com trail duplicação
    g.nodes[n]['pvalor_closeness'].append(stats.ttest_ind(geoTrue[n][0], trailFalse[n][0])[1])
    g.nodes[n]['pvalor_betweenness'].append(stats.ttest_ind(geoTrue[n][1],trailFalse[n][1])[1])
    #Comparando default (geodésica transferência) com walk transferência
    g.nodes[n]['pvalor_closeness'].append(stats.ttest_ind(geoTrue[n][0],walkTrue[n][0])[1])
    g.nodes[n]['pvalor_betweenness'].append(stats.ttest_ind(geoTrue[n][1],walkTrue[n][1])[1])
    #Comparando default (geodésica transferência) com walk duplicação
    g.nodes[n]['pvalor_closeness'].append(stats.ttest_ind(geoTrue[n][0],walkFalse[n][0])[1])
    g.nodes[n]['pvalor_betweenness'].append(stats.ttest_ind(geoTrue[n][1],walkFalse[n][1])[1])

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning:

numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88



In [19]:
#g.nodes[família]['pvalor_closeness'][qual comparação]
print(g.nodes[0]['pvalor_closeness'][4])

5.810654518680751e-158
0.14457174605201806


In [14]:
#Comparação dos Closeness
pd.DataFrame({
    'família': [g.nodes[n]['label'] for n in g.nodes],
    'Geodésica Duplicação': [g.nodes[n]['pvalor_closeness'][0] for n in g.nodes],
    'Path Transferência': [g.nodes[n]['pvalor_closeness'][1] for n in g.nodes],
    'Path Duplicação': [g.nodes[n]['pvalor_closeness'][2] for n in g.nodes],
    'Trail Transferência': [g.nodes[n]['pvalor_closeness'][3] for n in g.nodes],
    'Trail Duplicação': [g.nodes[n]['pvalor_closeness'][4] for n in g.nodes],
    'Walk Transferência': [g.nodes[n]['pvalor_closeness'][5] for n in g.nodes],
    'Walk Duplicação': [g.nodes[n]['pvalor_closeness'][6] for n in g.nodes],
})

,Geodésica Duplicação,Path Duplicação,Path Transferência,Trail Duplicação,Trail Transferência,Walk Duplicação,Walk Transferência,família
0,1,8.638211e-154,1.066981e-113,5.810655e-158,1.456091e-106,1.724785e-165,3.317655e-237,ginori
1,1,9.547386e-151,5.059008e-108,4.128907e-154,3.027475e-100,1.980906e-165,6.411966e-239,lambertes
2,1,9.875333e-180,2.032871e-120,1.203436e-189,2.422277e-135,4.972123e-195,8.157490e-292,albizzi
3,1,1.838828e-169,1.111209e-127,4.969998e-186,1.236109e-111,6.733052e-186,1.323145e-279,guadagni
4,1,3.821060e-146,9.330473e-106,8.631676e-150,3.162784e-100,3.265781e-180,6.493358e-228,pazzi
5,1,1.755352e-187,1.846738e-101,3.528862e-160,1.080100e-109,1.745597e-195,9.699765e-272,salviati
6,1,4.848282e-192,3.180101e-127,3.576759e-193,4.047267e-124,1.405169e-203,2.399150e-309,medici
7,1,3.164085e-178,1.099954e-134,1.838642e-196,4.601516e-134,1.662893e-195,3.142359e-296,tornabuon
8,1,2.593083e-168,1.989526e-125,3.299474e-166,6.735354e-127,1.443696e-172,6.272957e-278,bischeri
9,1,1.607397e-176,1.383221e-125,2.530390e-189,3.402490e-138,3.694038e-190,1.741663e-279,ridolfi


In [15]:
#Comparação dos Betweenness
pd.DataFrame({
    'família': [g.nodes[n]['label'] for n in g.nodes],
    'Geodésica Duplicação': [g.nodes[n]['pvalor_betweenness'][0] for n in g.nodes],
    'Path Transferência': [g.nodes[n]['pvalor_betweenness'][1] for n in g.nodes],
    'Path Duplicação': [g.nodes[n]['pvalor_betweenness'][2] for n in g.nodes],
    'Trail Transferência': [g.nodes[n]['pvalor_betweenness'][3] for n in g.nodes],
    'Trail Duplicação': [g.nodes[n]['pvalor_betweenness'][4] for n in g.nodes],
    'Walk Transferência': [g.nodes[n]['pvalor_betweenness'][5] for n in g.nodes],
    'Walk Duplicação': [g.nodes[n]['pvalor_betweenness'][6] for n in g.nodes],
})

,Geodésica Duplicação,Path Duplicação,Path Transferência,Trail Duplicação,Trail Transferência,Walk Duplicação,Walk Transferência,família
0,NaN,2.783251e-195,NaN,1.706285e-193,NaN,4.845807e-160,1.406184e-135,ginori
1,NaN,2.217989e-198,NaN,6.455253e-202,NaN,1.007364e-155,2.289287e-139,lambertes
2,1.958706e-179,1.253710e-196,6.275731e-08,2.203912e-198,2.532796e-54,2.077494e-162,7.619669e-156,albizzi
3,6.526670e-234,5.760294e-232,4.856150e-104,6.008262e-206,5.575393e-118,6.968206e-173,3.365305e-152,guadagni
4,NaN,1.988135e-179,NaN,1.637874e-176,NaN,2.384532e-138,9.624061e-126,pazzi
5,0.000000e+00,8.432384e-186,1.000000e+00,6.601137e-185,1.000000e+00,9.742066e-155,1.157611e-142,salviati
6,7.483430e-294,1.002420e-203,1.402667e-97,4.282063e-211,3.421214e-117,1.614488e-178,5.134261e-172,medici
7,5.988595e-124,2.540695e-235,3.263934e-147,6.137585e-218,7.760438e-139,7.733017e-178,4.943077e-161,tornabuon
8,1.746224e-193,3.890741e-228,9.323263e-142,1.918403e-210,2.211858e-152,2.080987e-174,2.353411e-144,bischeri
9,2.422152e-151,8.894968e-250,5.658833e-151,8.721809e-213,2.211212e-140,5.215889e-180,1.070327e-160,ridolfi


In [16]:
#sort_values(): to sort pandas data frame by one or more columns
#sort_index(): to sort pandas data frame by row index
#describe() Function gives the mean, std and IQR values. It excludes character column and calculate summary statistics only for numeric columns


In [17]:
#Tratamento Nan betweenness - famílias 0, 1, 4, 10. Caminhos geoFalse, pathTrue, trailTrue
listinha = [0, 1, 4, 10]
for i in listinha:
    print(geoFalse[i][1])
    print(pathTrue[i][1])
    print(trailTrue[i][1])
    
#Tudo 0, por isso o p-valor deu Nan.

#Tratamento Nan closenness - família 13. Caminho geoFalse
print(geoFalse[13][0])
print(geoTrue[13][0])
print(g.nodes[13]['pvalor_closeness'][0])
print(stats.ttest_ind(geoTrue[13][0],geoFalse[13][0]))

#Sem motivo aparente para o p-valor ser Nan já que todas as simulações deram valores maiores que 0 e menores que 1.

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [40]:
#Conferindo centralidades e p-valores da família salviati, já que obteve p-valores com iguais a 1.
print(g.nodes[5]['pvalor_betweenness'][1])
print(g.nodes[5]['pvalor_betweenness'][3])
print(g.nodes[5]['pvalor_betweenness'][5])
print(g.nodes[5]['mean_betweenness'])
print(g.nodes[5]['mean_closeness'])

1.0
1.0
1.1576107956109643e-142
1.809835164835165
0.14389608735939582


In [30]:
bet = pd.DataFrame({
        'família': [g.nodes[n]['label'] for n in g.nodes],
        'média closeness': [g.nodes[n]['mean_closeness'] for n in g.nodes],
        'média betweenness': [g.nodes[n]['mean_betweenness'] for n in g.nodes]})
bet
#b = bet.sort_values(by='mean_closeness')
#c = bet.sort_values(by='mean_betweeness')

,família,média betweenness,média closeness
0,ginori,0.799505,0.144572
1,lambertes,0.689451,0.145010
2,albizzi,3.001154,0.158868
3,guadagni,4.457857,0.164891
4,pazzi,0.767802,0.129178
5,salviati,1.809835,0.143896
6,medici,6.869396,0.167836
7,tornabuon,2.395330,0.172916
8,bischeri,2.732527,0.158500
9,ridolfi,2.514341,0.169875
